In [78]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use('ggplot')
import nltk

In [79]:
df = pd.read_csv('/content/test.csv', encoding='latin-1')
df

,textID,text,sentiment,Time of Tweet,Age of User,Country,Population -2020,Land Area (Km²),Density (P/Km²)
0,f87dea47db,Last session of the day http://twitpic.com/67ezh,neutral,morning,0-20,Afghanistan,38928346.0,652860.0,60.0
1,96d74cb729,Shanghai is also really exciting (precisely -...,positive,noon,21-30,Albania,2877797.0,27400.0,105.0
2,eee518ae67,"Recession hit Veronique Branquinho, she has to...",negative,night,31-45,Algeria,43851044.0,2381740.0,18.0
3,01082688c6,happy bday!,positive,morning,46-60,Andorra,77265.0,470.0,164.0
4,33987a8ee5,http://twitpic.com/4w75p - I like it!!,positive,noon,60-70,Angola,32866272.0,1246700.0,26.0
...,...,...,...,...,...,...,...,...,...
4810,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4811,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4812,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4813,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Data Cleaning

In [80]:
df.isnull().values.sum()

11529

In [81]:
df = df[['text', 'sentiment']]
df

,text,sentiment
0,Last session of the day http://twitpic.com/67ezh,neutral
1,Shanghai is also really exciting (precisely -...,positive
2,"Recession hit Veronique Branquinho, she has to...",negative
3,happy bday!,positive
4,http://twitpic.com/4w75p - I like it!!,positive
...,...,...
4810,NaN,NaN
4811,NaN,NaN
4812,NaN,NaN
4813,NaN,NaN


In [82]:
df = df.dropna()
df

,text,sentiment
0,Last session of the day http://twitpic.com/67ezh,neutral
1,Shanghai is also really exciting (precisely -...,positive
2,"Recession hit Veronique Branquinho, she has to...",negative
3,happy bday!,positive
4,http://twitpic.com/4w75p - I like it!!,positive
...,...,...
3529,"its at 3 am, im very tired but i can`t sleep ...",negative
3530,All alone in this old house again. Thanks for...,positive
3531,I know what you mean. My little dog is sinkin...,negative
3532,_sutra what is your next youtube video gonna b...,positive


In [83]:
import re
import nltk

nltk.download('stopwords')

from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

all_stopwords = stopwords.words('english')
all_stopwords.remove('not')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [84]:
corpus=[]

for i in range(0, 1000):
  review = re.sub('[^a-zA-Z]', ' ', df['text'][i])
  review = review.lower()
  review = review.split()
  review = [ps.stem(word) for word in review if not word in set(all_stopwords)]
  review = ' '.join(review)
  corpus.append(review)

# Data transformation

In [85]:
# loading bow dictionary
from sklearn.feature_extraction.text import CountVectorizer
import pickle
cvFile = '/content/bow.pkl'

cv = pickle.load(open(cvFile, "rb"))

In [86]:
X_fresh = cv.transform(corpus).toarray()
X_fresh.shape

(1000, 3534)

# Predictions

In [87]:
import joblib
classifier = joblib.load('/content/classifier.pkl')

In [88]:
y_pred = classifier.predict(X_fresh)
print(y_pred)

['negative' 'positive' 'neutral' 'negative' 'negative' 'positive'
 'negative' 'negative' 'neutral' 'neutral' 'negative' 'positive' 'neutral'
 'negative' 'negative' 'negative' 'negative' 'negative' 'positive'
 'neutral' 'negative' 'negative' 'negative' 'negative' 'negative'
 'negative' 'neutral' 'neutral' 'negative' 'negative' 'positive' 'neutral'
 'neutral' 'negative' 'negative' 'negative' 'positive' 'neutral'
 'negative' 'negative' 'negative' 'neutral' 'neutral' 'neutral' 'positive'
 'positive' 'negative' 'positive' 'negative' 'negative' 'positive'
 'negative' 'negative' 'negative' 'negative' 'positive' 'positive'
 'negative' 'neutral' 'positive' 'negative' 'neutral' 'neutral' 'negative'
 'neutral' 'negative' 'positive' 'neutral' 'negative' 'neutral' 'neutral'
 'neutral' 'neutral' 'negative' 'negative' 'negative' 'negative'
 'negative' 'positive' 'positive' 'positive' 'negative' 'negative'
 'positive' 'negative' 'neutral' 'negative' 'negative' 'negative'
 'neutral' 'negative' 'positiv

In [89]:
print(len(y_pred))
print(len(df))


1000
3534


In [92]:
df.to_csv('predicted_labels.csv', index=False)

In [91]:
if len(y_pred) < len(df):
    df['predicted_label'] = y_pred.tolist() + [None] * (len(df) - len(y_pred))
elif len(y_pred) > len(df):
    df['predicted_label'] = y_pred[:len(df)].tolist()
else:
    df['predicted_label'] = y_pred.tolist()


print(df.iloc[5:11])

                                                 text sentiment predicted_label
5                     that`s great!! weee!! visitors!  positive        positive
6             I THINK EVERYONE HATES ME ON HERE   lol  negative        negative
7    soooooo wish i could, but im in school and my...  negative        negative
8    and within a short time of the last clue all ...   neutral         neutral
9    What did you get?  My day is alright.. haven`...   neutral         neutral
10  My bike was put on hold...should have known th...  negative        negative


<ipython-input-91-7637a32f9d95>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['predicted_label'] = y_pred.tolist() + [None] * (len(df) - len(y_pred))
